In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.layers import Dropout, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


### 데이터 불러오기
* Kaggle의 필터링 된 버전의 Dogs vs Cats 데이터 세트를 사용

In [2]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
print(PATH)

68608000/68606236 [==============================] - 26s 0us/step
C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered


In [4]:
train_dir = os.path.join(PATH, 'train')            # 학습용
val_dir = os.path.join(PATH, 'validation')  # 평가용
print(train_dir)
print(val_dir)

C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\train
C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation


In [7]:
# 개, 고양이 학습, 평가 폴더 경로
print("개 : ", train_dir)
print("고양이 : ", val_dir)

# 개, 고양이 평가용 폴더 경로 
val_dogs_dir = os.path.join(val_dir, 'dogs')  # directory with our validation dog pictures
val_cats_dir = os.path.join(val_dir, 'cats')  # directory with our validation cat pictures

print("개 : ", val_dogs_dir)
print("고양이 : ", val_cats_dir)

개 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\train
고양이 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation
개 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation\dogs
고양이 :  C:\Users\seoul it\.keras\datasets\cats_and_dogs_filtered\validation\cats


In [8]:
# Initialising the CNN
# Step 1 - Convolution
# Step 2 - Pooling

model = Sequential()

# Step 1 - Convolution
#   컨볼루션 3x3  32필터, 입력이미지 64x64x3
model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
#   Pooling 필터 2x2, MaxPooling2D
model.add(MaxPooling2D(pool_size = (2, 2)))

# 두번째 Convolutional 계층 추가
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
model.add(Flatten())

# Step 4 - Full connection
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 1, activation = 'sigmoid'))  # 개고양이 분류

# Compiling the CNN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### ImageDataGenerator를 통한 학습용, 평가용 데이터 셋 만들기

In [9]:
from keras.preprocessing.image import ImageDataGenerator

* rescale : 기본값 없음. 다른 변환을 제공된 값으로 데이터를 곱함.
* zoom_range : 부동 소수점.[하한, 상한] 무작위 줌의 범위 [1-zoom_range, 1+zoom_range]
* horizontal_flip : 인풋을 무작위로 가로로 뒤집기

In [11]:
train_datagen = ImageDataGenerator(rescale= 1./255,  # 크기 재조절.
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)


### 데이터 가져오기
 * flow_from_directory 메소드를 사용하여 폴더 구조를 그대로 가져와, ImageDataGenerator 객체의 실제 데이터를 채워준다.

In [15]:
training_set = train_datagen.flow_from_directory(train_dir,
                                            target_size=(64,64),
                                            batch_size=32,
                                            class_mode='binary')

test_set = test_datagen.flow_from_directory(val_dir,
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### 모델 훈련

In [ ]:
model.fit_generator(training_set,
                   steps_per_epoch = 300,
                   epochs=25,
                   validation_data = test_set,
                   validation_steps = 2000)

Epoch 1/25
300/300 [==============================] - 279s 929ms/step - loss: 0.6363 - accuracy: 0.6273 - val_loss: 0.5774 - val_accuracy: 0.6549 - loss: 0.7462 - accuracy: 0.48 - ETA: 1:01 - loss: 
Epoch 2/25
300/300 [==============================] - 250s 833ms/step - loss: 0.5317 - accuracy: 0.7340 - val_loss: 0.8698 - val_accuracy: 0.7319
Epoch 3/25
300/300 [==============================] - 257s 856ms/step - loss: 0.4649 - accuracy: 0.7762 - val_loss: 0.5934 - val_accuracy: 0.71800.77 - ETA: 36s - loss: 0.4750 - accuracy: 0. - ETA: 36s - loss: 0.4734 - accuracy: 0.7 - ETA: 35s - loss: 0.4732 - accuracy: 0.77 - ETA: 35s - loss: 0.4729 - accuracy: 0 - ETA: 34s - loss: 0.4731 - accuracy: 0.7 - ETA: 33s - loss: 0.4733 - accu - ETA: 31s - loss: 0.4749  - ETA: 28s - loss: 0.4704 - ac - ETA: 26s - loss: 0.4725  - ETA: 22s - loss: 0.4737 - accuracy: 0.7 - ETA:
Epoch 4/25
300/300 [==============================] - 263s 877ms/step - loss: 0.4133 - accuracy: 0.8116 - val_loss: 1.3484 - val_a

### 모델 예측

* 배치 단위로 생성한 데이터에 대해 모델을 학습시킨다.

In [ ]:
output = model.predict_generator(test_set, steps=5)
print(test_set.class_indices)
print(output)